In [1]:
import os
from pathlib import Path
import sys
import os
import _pickle as pickle
import json
import copy 
import pandas
import numpy as np
import matplotlib.pyplot as plt 
from matplotlib.backends.backend_pdf import PdfPages
import csv
import json
from collections import Counter
import statistics 
import plotly.graph_objects as go
import math

In [2]:
from itertools import combinations
from scipy.stats import ttest_ind
import decimal

# create a new context for this task
ctx = decimal.Context()

# 20 digits should be enough for everyone :D
ctx.prec = 20

def float_to_str(f):
    """
    Convert the given float to a string,
    without resorting to scientific notation
    """
    d1 = ctx.create_decimal(repr(f))
    return format(d1, 'f')

In [3]:
with open('../../responseJudgement.json') as json_file:
    judgements = json.load(json_file)
    
with open('../../responseJudgement_sugg.json') as json_file:
    suggJudgements = json.load(json_file)

In [4]:
len(judgements)

15417

In [5]:
len(suggJudgements)

1014

In [6]:
cleanPath = '../../clean/'
settingName = 'Setting 2'

In [7]:
responseSet = {}

for root, dirs, files in os.walk(cleanPath):
    for file in files:
        if file.endswith(".json"):
            fileName = os.path.join(root, file)
            with open(fileName) as json_file:
                try:
                    data = json.load(json_file)
                    currFileName = data['currentFileName']
                    dialogs = data['dialogs']
                    
                    for i in range(1, len(dialogs)):
                        context = dialogs[i]['context']
                        uniqueStr = 'fileIndex=' + str(context['fileIndex']) + '&&contextLen=' + str(context['contextLen']) + '&&userResponse=' + context['userResponse'].strip()
                        
                        if(uniqueStr in responseSet):
                            continue
                        
                        else:
                            
                            foundMatches = [x for x in judgements if x['Input.contextLen'] == str(context['contextLen']) and x['Input.fileIndex'] == str(context['fileIndex'])]
                            foundJudgement = [x for x in foundMatches if x['Input.fileName'] == currFileName and x['Input.localIndex'] == str(context['localIndex'])]
                            
                            if(len(foundJudgement) > 0):
                                responseSet[uniqueStr] = foundJudgement
                        
                except Exception as e: print(e)

In [8]:
len(responseSet)

5139

In [9]:
suggSet = {}

for root, dirs, files in os.walk(cleanPath):
    for file in files:
        if file.endswith(".json"):
            fileName = os.path.join(root, file)
            with open(fileName) as json_file:
                try:
                    data = json.load(json_file)
                    currFileName = data['currentFileName']
                    dialogs = data['dialogs']
                    
                    for i in range(1, len(dialogs)):
                        context = dialogs[i]['context']
                        suggestions = context['suggestions']
                        
                        for sugg in suggestions:
                            uniqueStr = 'fileIndex=' + str(context['fileIndex']) + '&&contextLen=' + str(context['contextLen']) + '&&userResponse=' + sugg.strip()

                            if(uniqueStr in responseSet or uniqueStr in suggSet):
                                continue

                            else:

                                foundMatches = [x for x in suggJudgements if x['Input.contextLen'] == str(context['contextLen']) and x['Input.fileIndex'] == str(context['fileIndex'])]
                                foundJudgement = [x for x in foundMatches if x['Input.fileName'] == currFileName and x['Input.localIndex'] == str(context['localIndex'])]

                                if(len(foundJudgement) > 0):
                                    suggSet[uniqueStr] = foundJudgement
                        
                except Exception as e: print(e)

In [10]:
len(suggSet)

458

In [11]:
Qualtype = 'Interestingness'
metric = {
    'Answer.' + Qualtype +'.Strongly Disagree': 1,
    'Answer.' + Qualtype +'.Disagree': 2,
    'Answer.' + Qualtype +'.Neutral': 3,
    'Answer.' + Qualtype +'.Agree': 4,
    'Answer.' + Qualtype +'.Strongly Agree': 5
}

In [12]:
scoresHuman = []

for root, dirs, files in os.walk(cleanPath + 'Setting 1'):
    for file in files:
        if file.endswith(".json"):
            fileName = os.path.join(root, file)
            with open(fileName) as json_file:
                try:
                    data = json.load(json_file)
                    dialogs = data['dialogs']
                    
                    for i in range(1, len(dialogs)):
                        context = dialogs[i]['context']
                        
                        userResponse = context['userResponse']
                        fileIndex = context['fileIndex']
                        contextLen = context['contextLen']
                        
                        identifierStr = 'fileIndex=' + str(fileIndex) + '&&contextLen=' + str(contextLen) + '&&userResponse=' + userResponse.strip()
                        
                        responseJudgements =  responseSet[identifierStr]
                        
                        currFeedback = []
                        for feedback in responseJudgements:
                            for key in feedback:
                                
                                if('Answer.' + Qualtype in key and feedback[key] == 'true'):
                                    currFeedback.append(metric[key])
                        
                        scoresHuman.append(statistics.mean(currFeedback))
                            
                        
                    
                except Exception as e: print(e)                 

In [13]:
scoresBot = []

for root, dirs, files in os.walk(cleanPath + settingName):
    for file in files:
        if file.endswith(".json"):
            fileName = os.path.join(root, file)
            with open(fileName) as json_file:
                try:
                    data = json.load(json_file)
                    dialogs = data['dialogs']
                    
                    for i in range(1, len(dialogs)):
                        context = dialogs[i]['context']
                        
                        userResponse = context['userResponse']
                        fileIndex = context['fileIndex']
                        contextLen = context['contextLen']
                        suggestions = context['suggestions']
                        
                        identifierStr = 'fileIndex=' + str(fileIndex) + '&&contextLen=' + str(contextLen) + '&&userResponse=' + suggestions[0].strip()
                        
                        if(identifierStr in responseSet):
                            responseJudgements =  responseSet[identifierStr]
                        else:
                            responseJudgements =  suggSet[identifierStr]
                        
                        currFeedback = []
                        for feedback in responseJudgements:
                            for key in feedback:
                                
                                if('Answer.' + Qualtype in key and feedback[key] == 'true'):
                                    currFeedback.append(metric[key])
                        
                        scoresBot.append(statistics.mean(currFeedback))
                            
                        
                    
                except Exception as e: print(e)                 

In [14]:
sugg1 = []
sugg3 = []
sugg5 = []

autoSugg1 = []
autoSugg3 = []
autoSugg5 = []


for root, dirs, files in os.walk(cleanPath + settingName):
    for file in files:
        if file.endswith(".json"):
            fileName = os.path.join(root, file)
            with open(fileName) as json_file:
                try:
                    data = json.load(json_file)
                    dialogs = data['dialogs']
                    
                    for i in range(1, len(dialogs)):
                        context = dialogs[i]['context']
                        
                        userResponse = context['userResponse']
                        fileIndex = context['fileIndex']
                        contextLen = context['contextLen']
                        
                        identifierStr = 'fileIndex=' + str(fileIndex) + '&&contextLen=' + str(contextLen) + '&&userResponse=' + userResponse.strip()
                        
                        responseJudgements =  responseSet[identifierStr]
                        
                        currFeedback = []
                        
                        for feedback in responseJudgements:
                            for key in feedback:
                                
                                if('Answer.' + Qualtype in key and feedback[key] == 'true'):
                                    currFeedback.append(metric[key])
                        
                        
                        if(data['autoFilledSuggestion'] == 0):
                            
                            if(data['suggestionsNumber'] == 1):
                                sugg1.append(statistics.mean(currFeedback))

                            elif(data['suggestionsNumber'] == 3):
                                sugg3.append(statistics.mean(currFeedback))

                            elif(data['suggestionsNumber'] == 5):
                                sugg5.append(statistics.mean(currFeedback))
                        
                        else:

                            if(data['suggestionsNumber'] == 1):
                                autoSugg1.append(statistics.mean(currFeedback))

                            elif(data['suggestionsNumber'] == 3):
                                autoSugg3.append(statistics.mean(currFeedback))
                            
                            elif(data['suggestionsNumber'] == 5):
                                autoSugg5.append(statistics.mean(currFeedback))
                    
                    
                except Exception as e: print(e)

In [15]:
sugg1Bot = []
sugg3Bot = []
sugg5Bot = []

autoSugg1Bot = []
autoSugg3Bot = []
autoSugg5Bot = []


for root, dirs, files in os.walk(cleanPath + settingName):
    for file in files:
        if file.endswith(".json"):
            fileName = os.path.join(root, file)
            with open(fileName) as json_file:
                try:
                    data = json.load(json_file)
                    dialogs = data['dialogs']
                    
                    for i in range(1, len(dialogs)):
                        context = dialogs[i]['context']
                        
                        userResponse = context['userResponse']
                        fileIndex = context['fileIndex']
                        suggestions = context['suggestions']
                        
                        identifierStr = 'fileIndex=' + str(fileIndex) + '&&contextLen=' + str(contextLen) + '&&userResponse=' + suggestions[0].strip()
                        
                        if(identifierStr in responseSet):
                            responseJudgements =  responseSet[identifierStr]
                        else:
                            responseJudgements =  suggSet[identifierStr]
                        
                        currFeedback = []
                        
                        for feedback in responseJudgements:
                            for key in feedback:
                                
                                if('Answer.' + Qualtype in key and feedback[key] == 'true'):
                                    currFeedback.append(metric[key])
                        
                        
                        if(data['autoFilledSuggestion'] == 0):
                            
                            if(data['suggestionsNumber'] == 1):
                                sugg1Bot.append(statistics.mean(currFeedback))

                            elif(data['suggestionsNumber'] == 3):
                                sugg3Bot.append(statistics.mean(currFeedback))

                            elif(data['suggestionsNumber'] == 5):
                                sugg5Bot.append(statistics.mean(currFeedback))
                        
                        else:

                            if(data['suggestionsNumber'] == 1):
                                autoSugg1Bot.append(statistics.mean(currFeedback))

                            elif(data['suggestionsNumber'] == 3):
                                autoSugg3Bot.append(statistics.mean(currFeedback))
                            
                            elif(data['suggestionsNumber'] == 5):
                                autoSugg5Bot.append(statistics.mean(currFeedback))
                    
                    
                except Exception as e: print(e)

'fileIndex=18&&contextLen=3&&userResponse=Del Taco is an American fast food restaurant chain which specializes in American-style Mexican cuisine as well as American foods such as burgers, fries, and shakes.'
'fileIndex=28&&contextLen=3&&userResponse=American literature is literature written or produced in the United States and its preceding colonies (for specific discussions of poetry and theater, see Poetry of the United States and Theater in the United States).'
"fileIndex=19&&contextLen=3&&userResponse=It became a worldwide hit after the release of their debut album, 'Pablo Honey' (1993)."
'fileIndex=6&&contextLen=3&&userResponse=Snapple is a brand of tea and juice drinks which is owned by Dr Pepper Snapple Group and based in Plano, Texas.'
'fileIndex=19&&contextLen=3&&userResponse=The violin typically has four strings tuned in perfect fifths, and is most commonly played by drawing a bow across its strings, though it can also be played by plucking the strings with the fingers (pizzi

'fileIndex=15&&contextLen=3&&userResponse=Keisha Rose Ebert (; born March 1, 1987; formerly stylized as Kenya) is an American singer, songwriter, actress and rapper.'
'fileIndex=24&&contextLen=3&&userResponse=A variety of imaging techniques such as X-ray radiography, ultrasound, computed tomography (CT), nuclear medicine including positron emission tomography (PET), and magnetic resonance imaging (MRI) are used to diagnose and/or treat diseases.'
'fileIndex=28&&contextLen=3&&userResponse=The story charts the emotional development of the protagonist, Elizabeth Bennett, who learns the error of making hasty judgments and comes to appreciate the difference between the superficial and the essential.'
'fileIndex=4&&contextLen=3&&userResponse=Star Trek is an American media franchise based on the science fiction television series created by Gene Roddenberry.'
'fileIndex=27&&contextLen=3&&userResponse=The Simpsons is an American animated sitcom created by Matt Greening for the Fox Broadcasting 

'fileIndex=6&&contextLen=3&&userResponse=Waiting staff are those who work at a restaurant or a bar, and sometimes in private homes, attending customers—supplying them with food and drink as requested.'
'fileIndex=17&&contextLen=3&&userResponse=A Luxury yacht (also super-yacht, large yacht and mega-yacht) is a very expensive, privately owned, professionally crewed sailing or motor yacht.'
"fileIndex=26&&contextLen=3&&userResponse=Gabrielle Bother 'Coco' Chanel (19 August 1883 – 10 January 1971) was a French fashion designer and businesswoman."
'fileIndex=2&&contextLen=3&&userResponse=With an estimated 2016 population of 8, 537, 673 distributed over a land area of about, New York City is also the most densely populated major city in the United States.'
'fileIndex=24&&contextLen=3&&userResponse=\'"is an animated television series produced by Hasbro as part of the\'My Little Pony\' toy franchise, which is tied in with the 2010 launch of dolls and play sets, and original programming for U. 

In [16]:
settings = {
    'H': scoresHuman,
    'Bot': scoresBot,
    'sugg1': sugg1,
    'sugg3': sugg3,
    'sugg5': sugg5,
    'autoSugg1': autoSugg1,
    'autoSugg3': autoSugg3,
    'autoSugg5': autoSugg5
}

for list1, list2 in combinations(settings.keys(), 2):
    t, p = ttest_ind(settings[list1], settings[list2], equal_var = False)
    print (list1, list2, float_to_str(p))

H Bot 0.020766133009346027
H sugg1 0.6043462523638877
H sugg3 0.44602934282369033
H sugg5 0.0855580522217788
H autoSugg1 0.00037647490965491906
H autoSugg3 0.00000019197399975254686
H autoSugg5 0.00007807647767654497
Bot sugg1 0.07379364758869628
Bot sugg3 0.17153470670195797
Bot sugg5 0.9893306517528668
Bot autoSugg1 0.012689530054282987
Bot autoSugg3 0.0000005847927171247986
Bot autoSugg5 0.0021257690807412817
sugg1 sugg3 0.7896557466934342
sugg1 sugg5 0.20197057067689134
sugg1 autoSugg1 0.0015233939689003567
sugg1 autoSugg3 0.000000828282820657181
sugg1 autoSugg5 0.00032469606803100273
sugg3 sugg5 0.3209315725909541
sugg3 autoSugg1 0.00534154518757244
sugg3 autoSugg3 0.000006394968664104504
sugg3 autoSugg5 0.0012974744847772102
sugg5 autoSugg1 0.10784968873299865
sugg5 autoSugg3 0.0008413297786923649
sugg5 autoSugg5 0.0380976547893871
autoSugg1 autoSugg3 0.03302682880272516
autoSugg1 autoSugg5 0.5334168879524751
autoSugg3 autoSugg5 0.1516800613197893


In [19]:
colors = ['#FAEBD7', '#FFA07A', '#B0C4DE', '#AFEEEE', '#8FBC8F']
x = ['1 Suggestion', '3 Suggestions', '5 Suggestions']
fig = go.Figure()

xHuman = ['']
yHuman = [scoresHuman]
yHumanErorr = [np.std(z)/math.sqrt(len(z)) for z in yHuman] 
yHuman = [round(statistics.mean(z), 2) for z in yHuman]

fig.add_trace(go.Bar(
    name='Human Only',
    x=xHuman, y=yHuman, text=yHuman, textposition='inside',insidetextanchor='middle',
    marker_color='#B0C4DE',
    marker_line_color='rgb(8,48,107)', marker_line_width=1.5, opacity=0.8,
    error_y=dict(type='data', array=yHumanErorr)
))

xBot = ['']
yBot = [scoresBot]
yBotErorr = [np.std(z)/math.sqrt(len(z)) for z in yBot] 
yBot = [round(statistics.mean(z), 2) for z in yBot]

fig.add_trace(go.Bar(
    name='Chatbot Only',
    x=xBot, y=yBot, text=yBot, textposition='inside',insidetextanchor='middle',
    marker_color='#A9A9A9',
    marker_line_color='rgb(8,48,107)', marker_line_width=1.5, opacity=0.8,
    error_y=dict(type='data', array=yBotErorr)
))


ySuggOnly = [sugg1, sugg3, sugg5]
ySuggOnlyErorr = [np.std(z)/math.sqrt(len(z)) for z in ySuggOnly] 
ySuggOnly = [round(statistics.mean(z), 2) for z in ySuggOnly]

fig.add_trace(go.Bar(
    name='C1',
    x=x, y=ySuggOnly, text=ySuggOnly, textposition='inside',insidetextanchor='middle',
    marker_color='lightsalmon',
    marker_line_color='rgb(8,48,107)', marker_line_width=1.5, opacity=0.8,
    error_y=dict(type='data', array=ySuggOnlyErorr)
))

ySuggOnlyBot = [sugg1Bot, sugg3Bot, sugg5Bot]
ySuggOnlyBotErorr = [np.std(z)/math.sqrt(len(z)) for z in ySuggOnlyBot] 
ySuggOnlyBot = [round(statistics.mean(z), 2) for z in ySuggOnlyBot]

fig.add_trace(go.Bar(
    name='C1 - Chatbot Only',
    x=x, y=ySuggOnlyBot, text=ySuggOnlyBot, textposition='inside',insidetextanchor='middle',
    marker_color='#ffdacc',
    marker_line_color='rgb(8,48,107)', marker_line_width=1.5, opacity=0.8,
    error_y=dict(type='data', array=ySuggOnlyBotErorr)
))


yAutoSugg = [autoSugg1, autoSugg3, autoSugg5]
yAutoSuggErorr = [np.std(z)/math.sqrt(len(z)) for z in yAutoSugg] 
yAutoSugg = [round(statistics.mean(z), 2) for z in yAutoSugg]

fig.add_trace(go.Bar(
    name='C2',
    x=x, y=yAutoSugg, text=yAutoSugg, textposition='inside',insidetextanchor='middle',
    marker_color='#AFEEEE',
    marker_line_color='rgb(8,48,107)', marker_line_width=1.5, opacity=0.8,
    error_y=dict(type='data', array=yAutoSuggErorr)
))


yAutoSuggBot = [autoSugg1Bot, autoSugg3Bot, autoSugg5Bot]
yAutoSuggBotErorr = [np.std(z)/math.sqrt(len(z)) for z in yAutoSuggBot] 
yAutoSuggBot = [round(statistics.mean(z), 2) for z in yAutoSuggBot]

fig.add_trace(go.Bar(
    name='C2 - Chatbot Only',
    x=x, y=yAutoSuggBot, text=yAutoSuggBot, textposition='inside',insidetextanchor='middle',
    marker_color='#eafbfb',
    marker_line_color='rgb(8,48,107)', marker_line_width=1.5, opacity=0.8,
    error_y=dict(type='data', array=yAutoSuggBotErorr)
))


fig.update_layout(
    font=dict(
        size=13
    )
)
fig.update_yaxes(range=[3, 4.2])

fig.update_layout(legend_orientation="h")
fig.show()